In [1]:
import numpy as np
import shfl

def get_binary_vector_with_mean(mean, size):
    array_ones = np.ones((int(mean * size)))
    array_zeros = np.zeros((size - (int(mean * size))))
    array = np.concatenate((array_ones, array_zeros), axis = 0)
    np.random.shuffle(array)
    return array

Using TensorFlow backend.


In [2]:
array = get_binary_vector_with_mean(0.6, 10000)
print(len(array))
print(np.mean(array))
array.shape

10000
0.6


(10000,)

In [3]:
from shfl.core.data import UnprotectedAccess
from shfl.core.data import DataAccessDefinition
from shfl.core.dp_mechanism import RandomizeBinaryProperty

def diff_privacy_mean_test(mean, data_size):
    noise = 0.2
    mean_noise_distribution = 0.2
    array = get_binary_vector_with_mean(mean, data_size)
    federated_array = shfl.core.federated_operation.federate_array("my_federated_array", array, data_size)
    data_access_definition = DataAccessDefinition(dp_mechanism=RandomizeBinaryProperty(noise, mean_noise_distribution))
    federated_array.configure_data_access(data_access_definition)
    result = federated_array.query()
    
    noise_removed_result = (np.mean(result) - mean_noise_distribution*(noise))/(1-noise)
    
    print("Generated binary vector with mean: " + str(np.mean(array)))
    print("Differencial query mean result: " + str(np.mean(result)))
    print("Differencial query mean result: " + str(mean*(1-noise) + mean_noise_distribution * noise))
    print("Differencial query removing noise: " + str(noise_removed_result))
    print ("\n")

In [4]:
diff_privacy_mean_test(0.8, 10)
diff_privacy_mean_test(0.8, 100)
diff_privacy_mean_test(0.8, 1000)
diff_privacy_mean_test(0.8, 10000)
diff_privacy_mean_test(0.8, 100000)
diff_privacy_mean_test(0.8, 1000000)

Generated binary vector with mean: 0.8
Differencial query mean result: 0.6
Differencial query mean result: 0.6800000000000002
Differencial query removing noise: 0.6999999999999998


Generated binary vector with mean: 0.8
Differencial query mean result: 0.67
Differencial query mean result: 0.6800000000000002
Differencial query removing noise: 0.7875


Generated binary vector with mean: 0.8
Differencial query mean result: 0.688
Differencial query mean result: 0.6800000000000002
Differencial query removing noise: 0.8099999999999998


Generated binary vector with mean: 0.8
Differencial query mean result: 0.674
Differencial query mean result: 0.6800000000000002
Differencial query removing noise: 0.7925


Generated binary vector with mean: 0.8
Differencial query mean result: 0.67884
Differencial query mean result: 0.6800000000000002
Differencial query removing noise: 0.7985499999999999


Generated binary vector with mean: 0.8
Differencial query mean result: 0.680108
Differencial query mean r